In [23]:
import random
import json_lines
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import KeyedVectors

LIMIT_INPUT_ROWS= 200
MIN_ENGLISH= 0.0
WNL = WordNetLemmatizer()
REMOVEABLE_CHARS= '[^A-Za-z ]'
ALL_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
ACCEPTABLE_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MIN_WORD_OCCURRENCES= 5
VEC_SIZE= 5
MAX_GRAMS= 4
AMOUNT_OF_FILTS= 100

number_entries= 0
with open ('reviews.json', 'rb') as f:
    for item in json_lines.reader(f):
        number_entries+= 1

In [2]:
# READ IN INFO
reviews= []; polarity= []; are_early= []
included_indexs= []
j= 0
while j < LIMIT_INPUT_ROWS:
    randint= random.randint(0, number_entries-1)
    if not randint in included_indexs:
        included_indexs.append(randint)
        j+= 1
    
with open('reviews.json', 'rb') as f: # 5,000 entries in reviews.json
    for counter, item in enumerate(json_lines.reader(f)):
        if counter in included_indexs:
            reviews.append(item['text'])
            polarity.append(item['voted_up'])
            are_early.append(item['early_access'])

# END PRODUCTS: reviews, polarity, are_early

In [3]:
# FILTER TEXT BY CHARACTER

import re
new_reviews= []
for i, text in enumerate(reviews):
    text= re.sub(REMOVEABLE_CHARS, '', text)
    if not str.isspace(text) and text != "":
        new_reviews.append(text)
#     else:
#         print("removed ["+text+"]")
# print(reviews)
# print("########################################################################")
# print(new_reviews)
reviews= new_reviews

# END PRODUCTS: reviews

In [4]:
# FILTER TEXT BY LANGUAGE

filtered_reviews= []
filtered_polarity= []
filtered_are_early= []
for i, text in enumerate(reviews):
    try:
        langs = detect_langs(text)
    except:
        print("["+text+"]")
        print("ERROR: langs failed to define itself. Happens occassionally - just rerun")
        pass
    for lang in langs:
        if str(lang)[0:2] == 'en':
            if float(str(lang)[3:]) > MIN_ENGLISH:
                filtered_reviews.append(reviews[i])
                filtered_polarity.append(polarity[i])
                filtered_are_early.append(are_early[i])
reviews= filtered_reviews
polarity= filtered_polarity
are_early= filtered_are_early

#print(reviews)
# END PRODUCTS: reviews, polarity, are_early

In [5]:
# FILTER TEXT BY WORD TYPE

#print(reviews)
filtered_reviews= []
for i, text in enumerate(reviews):
    #print("-------------------------------------")
    new_text= []
    split_text= text.split()
    tags= nltk.pos_tag(split_text)
    #print(tags)
    #print("#############")
    lemd_split_text= []
    for word in split_text:
        lemd_split_text.append(WNL.lemmatize(word))
    tags= nltk.pos_tag(lemd_split_text)
    #print(tags)
    for j, word in enumerate(lemd_split_text):
        if tags[j][1] in ACCEPTABLE_TAGS:
            new_text.append(word)
    filtered_reviews.append(new_text)
reviews= [' '.join([str(elem) for elem in sublist]) for sublist in filtered_reviews]

# END PRODUCTS: reviews, polarity, are_early

In [6]:
# GET REVIEWS AS LIST OF LISTS

no_sentences= 0
reviews_as_ll= [] # ll denoting the list of list data
for text in reviews:
    split_text= text.split()
    reviews_as_ll.append(split_text)
    no_sentences+= 1

# END PRODUCTS: reviews_as_ll, no_sentences

In [7]:
for i, review in enumerate(reviews_as_ll):
    for j, word in enumerate(review):
        reviews_as_ll[i][j]= word.lower()

In [9]:
#X_train, X_test, y_train, y_test= train_test_split(reviews_as_ll, polarity, test_size=0.2, random_state=42)

In [10]:
model= gensim.models.Word2Vec(min_count= MIN_WORD_OCCURRENCES, vector_size=VEC_SIZE)
model.build_vocab(reviews_as_ll)
model.train(reviews_as_ll, total_examples=no_sentences, epochs=model.epochs) # can be a non-repeatable, 1-pass generator

(7829, 24260)

In [11]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

#vector = wv['the']  # Get numpy vector of a word

In [13]:
# print("####### GAME #######")
# vector = wv['amazing']
# print(vector)
# print("####### PLAYER #######")
# vector = wv['good']
# print(vector)

####### GAME #######
[-0.12886366  0.28580472  0.26098183 -0.4688893  -0.18429965]
####### PLAYER #######
[ 0.16223831  0.2709357   0.5825483  -0.6290702  -0.27084538]


In [24]:
# Make filters!

filters= []
for i in range(AMOUNT_OF_FILTS):
    filter= []
    n_gram= random.randint(2, MAX_GRAMS)
    for j in range(n_gram):
        filter_layer= []
        for k in range(VEC_SIZE):
            filter_layer.append(random.uniform(-1, 1))
        filter.append(filter_layer)
    filters.append(filter)

In [26]:
for filt in filters:
    print(filt)

[[0.4058640167312777, 0.6208053906290032, 0.016066502739414457, 0.41399093494335126, 0.25034199387967204], [-0.45240738790277946, -0.26730159735766024, -0.6615340326081942, 0.2537458690199008, -0.9112139008298967], [0.6850751198321203, -0.6309408998672219, 0.8421178587045053, 0.0742532252255641, -0.6298204127033482], [0.4469147182734472, 0.20795975024023905, 0.2624821745115129, 0.16524040493670622, 0.8055867918308017]]
[[-0.5193621497222731, 0.9790086946593137, -0.8532533908824598, -0.40212076574759825, 0.17114811823531806], [-0.21091107472963633, 0.21619333001814978, 0.2720507535815553, -0.8453675358793429, 0.2331965721092384], [-0.6689159046759341, 0.8898389468100405, 0.19217319039701297, 0.6051253668519834, -0.28357708026327866], [-0.451545805338905, 0.08667730872504209, -0.5353029700853933, 0.49029478689727557, -0.02756494504614171]]
[[-0.15470427958214383, 0.4039508347522198, 0.6124184978261542, 0.5892633055771226, 0.5338599389159728], [0.666727516676622, 0.2506929797070083, -0.09